# SELLERBOARD

## US MARKET

In [2]:
import pandas as pd
import os
import sys
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

In [3]:
data_2024 = pd.read_excel('Lifecycle/NewEleven_US_05_2018.xlsx')

scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Desktop/Keyword_Analysis/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1p3JWtN9HH1HxvARmx34Tmbt2iAjoYhK9fWIHgQr4PDE"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("Lifetime_US")

set_with_dataframe(sheet1, data_2024)

## CA MARKET

In [4]:
import pandas as pd
import os
import sys
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

In [5]:
data_2024 = pd.read_excel('Lifecycle/NewEleven_CA_02_2020.xlsx')

scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Desktop/Keyword_Analysis/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1p3JWtN9HH1HxvARmx34Tmbt2iAjoYhK9fWIHgQr4PDE"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("Lifetime_CA")

set_with_dataframe(sheet1, data_2024)

## UK MARKET

In [6]:
import pandas as pd
import os
import sys
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

In [7]:
data_2024 = pd.read_excel('Lifecycle/NewEleven_UK_01_2021.xlsx')

scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Desktop/Keyword_Analysis/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1p3JWtN9HH1HxvARmx34Tmbt2iAjoYhK9fWIHgQr4PDE"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("Lifetime_UK")

set_with_dataframe(sheet1, data_2024)

# BRAND ANALYTICS

## US MARKET

In [23]:
import pandas as pd
import os
import sys
import re
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

In [24]:
# ASIN, Impressions: Impressions, Clicks: Clicks, Clicks: Click Rate (CTR), Cart Adds: Cart Adds, Purchases: Purchases, Purchases: Search Traffic Sales, Purchases: Conversion Rate %, Reporting Date

# Các cột cần lấy
selected_columns = [
    "ASIN", 
    "Impressions: Impressions", 
    "Clicks: Clicks", 
    "Clicks: Click Rate (CTR)", 
    "Cart Adds: Cart Adds", 
    "Purchases: Purchases", 
    "Purchases: Search Traffic Sales", 
    "Purchases: Conversion Rate %",
    "Reporting Date"
]

def get_quarter_from_date(date_str):
    """
    Lấy quarter từ ngày tháng
    """
    try:
        # Thử các format ngày tháng phổ biến
        date_formats = ['%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y', '%Y/%m/%d']
        
        for fmt in date_formats:
            try:
                date_obj = datetime.strptime(str(date_str), fmt)
                quarter = (date_obj.month - 1) // 3 + 1
                return quarter
            except:
                continue
                
        # Nếu không parse được, thử pandas
        date_obj = pd.to_datetime(date_str)
        quarter = (date_obj.month - 1) // 3 + 1
        return quarter
    except:
        return None

def process_folder(base_path, year):
    """
    Xử lý tất cả file CSV trong folder của một năm
    """
    folder_path = os.path.join(base_path, str(year))
    
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} không tồn tại!")
        return pd.DataFrame()
    
    all_data = []
    
    # Lấy tất cả file CSV trong folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        
        try:
            # Đọc file CSV
            df = pd.read_csv(file_path, skiprows=1)
            
            # Kiểm tra xem các cột cần thiết có tồn tại không
            available_columns = [col for col in selected_columns if col in df.columns]
            missing_columns = [col for col in selected_columns if col not in df.columns]
            
            if missing_columns:
                print(f"File {file_name}: Thiếu các cột {missing_columns}")
            
            if available_columns:
                # Lấy dữ liệu từ các cột có sẵn
                df_selected = df[available_columns].copy()
                
                # Thêm cột Year
                df_selected['Year'] = year
                
                # Thêm cột Quarter nếu có cột Reporting Date
                if 'Reporting Date' in df_selected.columns:
                    df_selected['Quarter'] = df_selected['Reporting Date'].apply(get_quarter_from_date)
                else:
                    df_selected['Quarter'] = None
                
                
                all_data.append(df_selected)
                print(f"Đã xử lý file: {file_name} - {len(df_selected)} rows")
            else:
                print(f"File {file_name}: Không có cột nào phù hợp")
                
        except Exception as e:
            print(f"Lỗi khi đọc file {file_name}: {str(e)}")
    
    # Combine tất cả data
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()

def main():
    # Đường dẫn đến folder chứa các folder năm (cần thay đổi theo đường dẫn thực tế)
    # Đường dẫn gốc đến folder
    base_path = "/Users/thuytrinh/Desktop/ADS-Performance/US/BA_US"  # Thay đổi đường dẫn này theo folder thực tế của bạn
    
    # Xử lý data từ folder 2023 và 2024
    df_2023 = process_folder(base_path, 2023)
    df_2024 = process_folder(base_path, 2024)
    
    # Combine data từ cả 2 năm
    all_dataframes = []
    
    if not df_2023.empty:
        all_dataframes.append(df_2023)
        print(f"Data 2023: {len(df_2023)} rows")
    
    if not df_2024.empty:
        all_dataframes.append(df_2024)
        print(f"Data 2024: {len(df_2024)} rows")
    
    if all_dataframes:
        # Combine tất cả data
        final_df = pd.concat(all_dataframes, ignore_index=True)
        
        # Sắp xếp theo Year và Quarter
        final_df = final_df.sort_values(['Year', 'Quarter', 'Reporting Date'])
        
        print(f"\nTổng cộng: {len(final_df)} rows")
        print(f"Các cột trong data cuối cùng: {list(final_df.columns)}")
        
        # Hiển thị thông tin tổng quan
        print("\nThông tin tổng quan:")
        print(final_df.groupby(['Year', 'Quarter']).size().reset_index(name='Count'))
        
        # Lưu file kết quả
        # Đường dẫn thư mục lưu file
        output_folder = "/Users/thuytrinh/Desktop/ADS-Performance/Lifecycle/BA"

        # Tạo tên file kèm timestamp
        output_filename = f"combined_data_US.xlsx"

        # Ghép đường dẫn đầy đủ
        output_path = os.path.join(output_folder, output_filename)

        # Ghi file Excel
        final_df.to_excel(output_path, index=False)

        print(f"\n✅ Đã lưu file kết quả tại: {output_path}")
        
        # Hiển thị một vài dòng đầu
        print("\nDữ liệu mẫu:")
        print(final_df.head())
        
        return final_df
    else:
        print("Không có dữ liệu để xử lý!")
        return pd.DataFrame()

if __name__ == "__main__":
    result_df = main()


Đã xử lý file: US_Quarter 3_2023.csv - 196 rows
Đã xử lý file: US_Quarter 2_2023.csv - 165 rows
Đã xử lý file: US_Quarter 4_2023.csv - 255 rows
Đã xử lý file: US_Quarter 3_2024.csv - 273 rows
Đã xử lý file: US_Quarter 2_2024.csv - 284 rows
Đã xử lý file: US_Quarter 4_2024.csv - 356 rows
Đã xử lý file: US_Quarter 1_2024.csv - 271 rows
Data 2023: 616 rows
Data 2024: 1184 rows

Tổng cộng: 1800 rows
Các cột trong data cuối cùng: ['ASIN', 'Impressions: Impressions', 'Clicks: Clicks', 'Clicks: Click Rate (CTR)', 'Cart Adds: Cart Adds', 'Purchases: Purchases', 'Purchases: Search Traffic Sales', 'Purchases: Conversion Rate %', 'Reporting Date', 'Year', 'Quarter']

Thông tin tổng quan:
   Year  Quarter  Count
0  2023        2    165
1  2023        3    196
2  2023        4    255
3  2024        1    271
4  2024        2    284
5  2024        3    273
6  2024        4    356

✅ Đã lưu file kết quả tại: /Users/thuytrinh/Desktop/ADS-Performance/Lifecycle/BA/combined_data_US.xlsx

Dữ liệu mẫu:
    

In [25]:
ba_us_2025 = pd.read_excel('/Users/thuytrinh/Desktop/ADS-Performance/Lifecycle/BA/BA_US_2025.xlsx')

In [26]:
# Concat final_df with ba_us_2025
final_df = pd.concat([result_df, ba_us_2025], ignore_index=True)

In [27]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/thuytrinh/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1p3JWtN9HH1HxvARmx34Tmbt2iAjoYhK9fWIHgQr4PDE"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("Lifetime_US_BA")

sheet1.clear()  # Clear the existing data
set_with_dataframe(sheet1, final_df)

## CA MARKET

In [33]:
import pandas as pd
import os
import sys
import re
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

In [34]:
# ASIN, Impressions: Impressions, Clicks: Clicks, Clicks: Click Rate (CTR), Cart Adds: Cart Adds, Purchases: Purchases, Purchases: Search Traffic Sales, Purchases: Conversion Rate %, Reporting Date

# Các cột cần lấy
selected_columns = [
    "ASIN", 
    "Impressions: Impressions", 
    "Clicks: Clicks", 
    "Clicks: Click Rate (CTR)", 
    "Cart Adds: Cart Adds", 
    "Purchases: Purchases", 
    "Purchases: Search Traffic Sales", 
    "Purchases: Conversion Rate %",
    "Reporting Date"
]

def get_quarter_from_date(date_str):
    """
    Lấy quarter từ ngày tháng
    """
    try:
        # Thử các format ngày tháng phổ biến
        date_formats = ['%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y', '%Y/%m/%d']
        
        for fmt in date_formats:
            try:
                date_obj = datetime.strptime(str(date_str), fmt)
                quarter = (date_obj.month - 1) // 3 + 1
                return quarter
            except:
                continue
                
        # Nếu không parse được, thử pandas
        date_obj = pd.to_datetime(date_str)
        quarter = (date_obj.month - 1) // 3 + 1
        return quarter
    except:
        return None

def process_folder(base_path, year):
    """
    Xử lý tất cả file CSV trong folder của một năm
    """
    folder_path = os.path.join(base_path, str(year))
    
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} không tồn tại!")
        return pd.DataFrame()
    
    all_data = []
    
    # Lấy tất cả file CSV trong folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        
        try:
            # Đọc file CSV
            df = pd.read_csv(file_path, skiprows=1)
            
            # Kiểm tra xem các cột cần thiết có tồn tại không
            available_columns = [col for col in selected_columns if col in df.columns]
            missing_columns = [col for col in selected_columns if col not in df.columns]
            
            if missing_columns:
                print(f"File {file_name}: Thiếu các cột {missing_columns}")
            
            if available_columns:
                # Lấy dữ liệu từ các cột có sẵn
                df_selected = df[available_columns].copy()
                
                # Thêm cột Year
                df_selected['Year'] = year
                
                # Thêm cột Quarter nếu có cột Reporting Date
                if 'Reporting Date' in df_selected.columns:
                    df_selected['Quarter'] = df_selected['Reporting Date'].apply(get_quarter_from_date)
                else:
                    df_selected['Quarter'] = None
                
                
                all_data.append(df_selected)
                print(f"Đã xử lý file: {file_name} - {len(df_selected)} rows")
            else:
                print(f"File {file_name}: Không có cột nào phù hợp")
                
        except Exception as e:
            print(f"Lỗi khi đọc file {file_name}: {str(e)}")
    
    # Combine tất cả data
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()

def main():
    # Đường dẫn đến folder chứa các folder năm (cần thay đổi theo đường dẫn thực tế)
    # Đường dẫn gốc đến folder
    base_path = "/Users/thuytrinh/Desktop/ADS-Performance/CA/BA_CA"  # Thay đổi đường dẫn này theo folder thực tế của bạn
    
    # Xử lý data từ folder 2023 và 2024
    df_2023 = process_folder(base_path, 2023)
    df_2024 = process_folder(base_path, 2024)
    
    # Combine data từ cả 2 năm
    all_dataframes = []
    
    if not df_2023.empty:
        all_dataframes.append(df_2023)
        print(f"Data 2023: {len(df_2023)} rows")
    
    if not df_2024.empty:
        all_dataframes.append(df_2024)
        print(f"Data 2024: {len(df_2024)} rows")
    
    if all_dataframes:
        # Combine tất cả data
        final_df = pd.concat(all_dataframes, ignore_index=True)
        
        # Sắp xếp theo Year và Quarter
        final_df = final_df.sort_values(['Year', 'Quarter', 'Reporting Date'])
        
        print(f"\nTổng cộng: {len(final_df)} rows")
        print(f"Các cột trong data cuối cùng: {list(final_df.columns)}")
        
        # Hiển thị thông tin tổng quan
        print("\nThông tin tổng quan:")
        print(final_df.groupby(['Year', 'Quarter']).size().reset_index(name='Count'))
        
        # Lưu file kết quả
        # Đường dẫn thư mục lưu file
        output_folder = "/Users/thuytrinh/Desktop/ADS-Performance/Lifecycle/BA"

        # Tạo tên file kèm timestamp
        output_filename = f"combined_data_CA.xlsx"

        # Ghép đường dẫn đầy đủ
        output_path = os.path.join(output_folder, output_filename)

        # Ghi file Excel
        final_df.to_excel(output_path, index=False)

        print(f"\n✅ Đã lưu file kết quả tại: {output_path}")
        
        # Hiển thị một vài dòng đầu
        print("\nDữ liệu mẫu:")
        print(final_df.head())
        
        return final_df
    else:
        print("Không có dữ liệu để xử lý!")
        return pd.DataFrame()

if __name__ == "__main__":
    result_df = main()


Đã xử lý file: CA_Quarter 4_2023.csv - 183 rows
Đã xử lý file: CA_Quarter 2_2023.csv - 109 rows
Đã xử lý file: CA_Quarter 3_2023.csv - 140 rows
Đã xử lý file: CA_Quarter 4_2024.csv - 247 rows
Đã xử lý file: CA_Quarter 2_2024.csv - 199 rows
Đã xử lý file: CA_Quarter 3_2024.csv - 210 rows
Đã xử lý file: CA_Quarter 1_2024.csv - 188 rows
Data 2023: 432 rows
Data 2024: 844 rows

Tổng cộng: 1276 rows
Các cột trong data cuối cùng: ['ASIN', 'Impressions: Impressions', 'Clicks: Clicks', 'Clicks: Click Rate (CTR)', 'Cart Adds: Cart Adds', 'Purchases: Purchases', 'Purchases: Search Traffic Sales', 'Purchases: Conversion Rate %', 'Reporting Date', 'Year', 'Quarter']

Thông tin tổng quan:
   Year  Quarter  Count
0  2023        2    109
1  2023        3    140
2  2023        4    183
3  2024        1    188
4  2024        2    199
5  2024        3    210
6  2024        4    247

✅ Đã lưu file kết quả tại: /Users/thuytrinh/Desktop/ADS-Performance/Lifecycle/BA/combined_data_CA.xlsx

Dữ liệu mẫu:
     

In [35]:
ba_ca_2025 = pd.read_excel('/Users/thuytrinh/Desktop/ADS-Performance/Lifecycle/BA/BA_CA_2025.xlsx')

In [36]:
# Concat final_df with ba_us_2025
final_df = pd.concat([result_df, ba_ca_2025], ignore_index=True)

In [37]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/thuytrinh/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1p3JWtN9HH1HxvARmx34Tmbt2iAjoYhK9fWIHgQr4PDE"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("Lifetime_CA_BA")

sheet1.clear()  # Clear the existing data
set_with_dataframe(sheet1, final_df)

## UK MARKET

In [44]:
import pandas as pd
import os
import sys
import re
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

In [45]:
# ASIN, Impressions: Impressions, Clicks: Clicks, Clicks: Click Rate (CTR), Cart Adds: Cart Adds, Purchases: Purchases, Purchases: Search Traffic Sales, Purchases: Conversion Rate %, Reporting Date

# Các cột cần lấy
selected_columns = [
    "ASIN", 
    "Impressions: Impressions", 
    "Clicks: Clicks", 
    "Clicks: Click Rate (CTR)", 
    "Basket Adds: Basket Adds",
    "Purchases: Purchases", 
    "Purchases: Search Traffic Sales", 
    "Purchases: Conversion Rate %",
    "Reporting Date"
]

def get_quarter_from_date(date_str):
    """
    Lấy quarter từ ngày tháng
    """
    try:
        # Thử các format ngày tháng phổ biến
        date_formats = ['%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y', '%Y/%m/%d']
        
        for fmt in date_formats:
            try:
                date_obj = datetime.strptime(str(date_str), fmt)
                quarter = (date_obj.month - 1) // 3 + 1
                return quarter
            except:
                continue
                
        # Nếu không parse được, thử pandas
        date_obj = pd.to_datetime(date_str)
        quarter = (date_obj.month - 1) // 3 + 1
        return quarter
    except:
        return None

def process_folder(base_path, year):
    """
    Xử lý tất cả file CSV trong folder của một năm
    """
    folder_path = os.path.join(base_path, str(year))
    
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} không tồn tại!")
        return pd.DataFrame()
    
    all_data = []
    
    # Lấy tất cả file CSV trong folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        
        try:
            # Đọc file CSV
            df = pd.read_csv(file_path, skiprows=1)
            
            # Kiểm tra xem các cột cần thiết có tồn tại không
            available_columns = [col for col in selected_columns if col in df.columns]
            missing_columns = [col for col in selected_columns if col not in df.columns]
            
            if missing_columns:
                print(f"File {file_name}: Thiếu các cột {missing_columns}")
            
            if available_columns:
                # Lấy dữ liệu từ các cột có sẵn
                df_selected = df[available_columns].copy()
                
                # Thêm cột Year
                df_selected['Year'] = year
                
                # Thêm cột Quarter nếu có cột Reporting Date
                if 'Reporting Date' in df_selected.columns:
                    df_selected['Quarter'] = df_selected['Reporting Date'].apply(get_quarter_from_date)
                else:
                    df_selected['Quarter'] = None
                
                
                all_data.append(df_selected)
                print(f"Đã xử lý file: {file_name} - {len(df_selected)} rows")
            else:
                print(f"File {file_name}: Không có cột nào phù hợp")
                
        except Exception as e:
            print(f"Lỗi khi đọc file {file_name}: {str(e)}")
    
    # Combine tất cả data
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()

def main():
    # Đường dẫn đến folder chứa các folder năm (cần thay đổi theo đường dẫn thực tế)
    # Đường dẫn gốc đến folder
    base_path = "/Users/thuytrinh/Desktop/ADS-Performance/UK/BA_UK"  # Thay đổi đường dẫn này theo folder thực tế của bạn
    
    # Xử lý data từ folder 2023 và 2024
    df_2023 = process_folder(base_path, 2023)
    df_2024 = process_folder(base_path, 2024)
    df_2025 = process_folder(base_path, 2025)
    
    # Combine data từ cả 2 năm
    all_dataframes = []
    
    if not df_2023.empty:
        all_dataframes.append(df_2023)
        print(f"Data 2023: {len(df_2023)} rows")
    
    if not df_2024.empty:
        all_dataframes.append(df_2024)
        print(f"Data 2024: {len(df_2024)} rows")

    if not df_2025.empty:
        all_dataframes.append(df_2025)
        print(f"Data 2025: {len(df_2025)} rows")
    
    if all_dataframes:
        # Combine tất cả data
        final_df = pd.concat(all_dataframes, ignore_index=True)
        
        # Sắp xếp theo Year và Quarter
        final_df = final_df.sort_values(['Year', 'Quarter', 'Reporting Date'])
        
        print(f"\nTổng cộng: {len(final_df)} rows")
        print(f"Các cột trong data cuối cùng: {list(final_df.columns)}")
        
        # Hiển thị thông tin tổng quan
        print("\nThông tin tổng quan:")
        print(final_df.groupby(['Year', 'Quarter']).size().reset_index(name='Count'))
        
        # Lưu file kết quả
        # Đường dẫn thư mục lưu file
        output_folder = "/Users/thuytrinh/Desktop/ADS-Performance/Lifecycle/BA"

        # Tạo tên file kèm timestamp
        output_filename = f"combined_data_UK.xlsx"

        # Ghép đường dẫn đầy đủ
        output_path = os.path.join(output_folder, output_filename)

        # Ghi file Excel
        final_df.to_excel(output_path, index=False)

        print(f"\n✅ Đã lưu file kết quả tại: {output_path}")
        
        # Hiển thị một vài dòng đầu
        print("\nDữ liệu mẫu:")
        print(final_df.head())
        
        return final_df
    else:
        print("Không có dữ liệu để xử lý!")
        return pd.DataFrame()

if __name__ == "__main__":
    result_df = main()


Đã xử lý file: GB_Quarter 4_2023.csv - 119 rows
Đã xử lý file: GB_Quarter 3_2023.csv - 56 rows
Đã xử lý file: GB_Quarter 4_2024.csv - 98 rows
Đã xử lý file: GB_Quarter 3_2024.csv - 78 rows
Đã xử lý file: GB_Quarter 2_2024.csv - 139 rows
Đã xử lý file: GB_Quarter 1_2024.csv - 153 rows
Đã xử lý file: GB_Quarter 2_2025.csv - 45 rows
Đã xử lý file: GB_Quarter 1_2025.csv - 100 rows
Data 2023: 175 rows
Data 2024: 468 rows
Data 2025: 145 rows

Tổng cộng: 788 rows
Các cột trong data cuối cùng: ['ASIN', 'Impressions: Impressions', 'Clicks: Clicks', 'Clicks: Click Rate (CTR)', 'Basket Adds: Basket Adds', 'Purchases: Purchases', 'Purchases: Search Traffic Sales', 'Purchases: Conversion Rate %', 'Reporting Date', 'Year', 'Quarter']

Thông tin tổng quan:
   Year  Quarter  Count
0  2023        3     56
1  2023        4    119
2  2024        1    153
3  2024        2    139
4  2024        3     78
5  2024        4     98
6  2025        1    100
7  2025        2     45

✅ Đã lưu file kết quả tại: /Use

In [46]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/thuytrinh/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1p3JWtN9HH1HxvARmx34Tmbt2iAjoYhK9fWIHgQr4PDE"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("Lifetime_UK_BA")

sheet1.clear()  # Clear the existing data
set_with_dataframe(sheet1, result_df)